In [86]:
from pandas import Series, DataFrame
import numpy as np
import pandas as pnd
import imp

#Evaluation results for functionality diffusion based recommendation algorithm#

Average precision has been computed for k in range from 1 to 10. Where k is the number of retrieved documents. Documents are retrieved by relevance e.g. more relevant document is assumed to be retrieved ahead.
Evaluation has been done for 85 mashups published after 1st of January 2015.

Each mashup has been treated as a query i.e. we considered the set of categories of a given mashup as a user's query.

Average precision has been computed for each user's query for each k.

Each retrieved document (service) has been analized according to it's relevance to a given mashup. Recommended document has been treated as fully relevant if it is actually involved in a given mashup. Fully relevant document got a score 1. 
Also we used a notion of partially relevant documents. We considered a document as partially relevant in case if mashup's neighbour used recommended document. Mashup's neighbour is another mashup with similar set of categories. The score of partially relevant document was computed as cosine similarity between a given mashup and it's closest neighbour in terms of set of assigned categories.

Recommended document has been treated as irrelevant if it was not involved in a given mashup and it has not been used by any neighbours of a mashup under consideration. Irrelevant document got a score 0. 

In [357]:
method1_1_1 = pnd.read_csv("results/1_1_1_result.csv")
del method1_1_1["Unnamed: 0"]
mean111 = method1_1_1.mean(axis=0)

In [358]:
method1_1_0 = pnd.read_csv("results/1_1_0_result.csv")
del method1_1_0["Unnamed: 0"]
mean110 = method1_1_0.mean(axis=0)

In [389]:
method1_0_0 = pnd.read_csv("results/1_0_0_result.csv")
del method1_0_0["Unnamed: 0"]
mean100 = method1_0_0.mean(axis=0)

In [399]:
method1_0_1 = pnd.read_csv("results/1_0_1_result.csv")
del method1_0_1["Unnamed: 0"]
mean101 = method1_0_1.mean(axis=0)

In [400]:
mean_data_frame = DataFrame([mean111, mean110, mean100, mean101], index = ['Method 1 1 1', 'Method 1 1 0', 'Method 1 0 0', 'Method 1 0 1'])
mean_data_frame.columns = [['Mean Average Precision for 85 trasactions' for i in range(10)], ['k = %s' %k for k in range(1,11)]]
mean_data_frame

Mean Average Precision for 85 trasactions                      \
                                                 k = 1     k = 2     k = 3   
Method 1 1 1                                  0.313988  0.284756  0.271969   
Method 1 1 0                                  0.275182  0.246972  0.222994   
Method 1 0 0                                  0.387583  0.290034  0.247921   
Method 1 0 1                                  0.392470  0.284431  0.262084   

                                                                          \
                 k = 4     k = 5     k = 6     k = 7     k = 8     k = 9   
Method 1 1 1  0.264133  0.248883  0.240697  0.235652  0.229302  0.222265   
Method 1 1 0  0.215256  0.214968  0.209599  0.205567  0.200977  0.195354   
Method 1 0 0  0.226014  0.207222  0.199552  0.191371  0.188553  0.185387   
Method 1 0 1  0.243170  0.227907  0.222516  0.215552  0.210924  0.203968   

                        
                k = 10  
Method 1 1 1  0.189593  
Method 1 1 0  0.165344  
Method 1 0 0  0.144334  
Method 1 0 1  0.169687

##Evaluation results for recommendation algorithm, where weights for all three metrics is 1 (Method 1,1,1)##

In [351]:
method1_1_1.columns = [['Method: lambdaSC=1, lambdaRE=1, lambdaDT=1']+['Mean Average precision' for i in range(10)],
                       ['Mashup']+['k = %s' %k for k in range(1,11)]]
method1_1_1.mean(axis=0)

Mean Average precision  k = 1     0.313988
                        k = 2     0.284756
                        k = 3     0.271969
                        k = 4     0.264133
                        k = 5     0.248883
                        k = 6     0.240697
                        k = 7     0.235652
                        k = 8     0.229302
                        k = 9     0.222265
                        k = 10    0.189593
dtype: float64

In [341]:
method1_1_1.columns = [['Method: lambdaSC=1, lambdaRE=1, lambdaDT=1']+['Average precision' for i in range(10)],
                       ['Mashup']+['k = %s' %k for k in range(1,11)]]
method1_1_1

Method: lambdaSC=1, lambdaRE=1, lambdaDT=1 Average precision  \
                                               Mashup             k = 1   
0   http://www.programmableweb.com/mashup/geoloqi-...          0.250000   
1   http://www.programmableweb.com/mashup/collecti...          0.250000   
2   http://www.programmableweb.com/mashup/gemerald...          1.000000   
3     http://www.programmableweb.com/mashup/theonespy          0.333333   
4   http://www.programmableweb.com/mashup/going-ou...          0.083333   
5   http://www.programmableweb.com/mashup/ifixit-a...          0.055556   
6   http://www.programmableweb.com/mashup/idealist...          0.666667   
7   http://www.programmableweb.com/mashup/itavli-i...          1.000000   
8   http://www.programmableweb.com/mashup/grasshopper          0.000000   
9     http://www.programmableweb.com/mashup/nostalgia          0.250000   
10  http://www.programmableweb.com/mashup/maco-tra...          0.125000   
11    http://www.programmableweb.com/mashup/detective          1.000000   
12  http://www.programmableweb.com/mashup/early-mo...          0.333333   
13  http://www.programmableweb.com/mashup/serendip...          0.500000   
14  http://www.programmableweb.com/mashup/strava-s...          0.250000   
15   http://www.programmableweb.com/mashup/se-ranking          0.000000   
16  http://www.programmableweb.com/mashup/weather-...          0.000000   
17   http://www.programmableweb.com/mashup/stel-order          0.055556   
18  http://www.programmableweb.com/mashup/duplicat...          0.250000   
19  http://www.programmableweb.com/mashup/soccer-s...          0.125000   
20    http://www.programmableweb.com/mashup/findilike          0.250000   
21  http://www.programmableweb.com/mashup/windshie...          1.000000   
22  http://www.programmableweb.com/mashup/criminal...          0.111111   
23  http://www.programmableweb.com/mashup/slick-de...          0.166667   
24   http://www.programmableweb.com/mashup/usersthink          0.125000   
25  http://www.programmableweb.com/mashup/magento-...          0.500000   
26  http://www.programmableweb.com/mashup/moves-da...          0.500000   
27   http://www.programmableweb.com/mashup/touristica          0.050000   
28     http://www.programmableweb.com/mashup/dino-map          0.166667   
29  http://www.programmableweb.com/mashup/get-control          0.666667   
..                                                ...               ...   
55  http://www.programmableweb.com/mashup/99design...          0.333333   
56  http://www.programmableweb.com/mashup/bing-map...          0.333333   
57  http://www.programmableweb.com/mashup/data-vis...          0.000000   
58  http://www.programmableweb.com/mashup/click4su...          0.100000   
59     http://www.programmableweb.com/mashup/moonotes          0.500000   
60    http://www.programmableweb.com/mashup/athenaeum          0.055556   
61      http://www.programmableweb.com/mashup/segment          0.333333   
62  http://www.programmableweb.com/mashup/moves-el...          0.500000   
63  http://www.programmableweb.com/mashup/uber-cal...          0.083333   
64  http://www.programmableweb.com/mashup/geobytes...          0.000000   
65      http://www.programmableweb.com/mashup/oznewsy          0.125000   
66  http://www.programmableweb.com/mashup/weglore-...          0.250000   
67  http://www.programmableweb.com/mashup/intellin...          1.000000   
68     http://www.programmableweb.com/mashup/expin.me          0.125000   
69  http://www.programmableweb.com/mashup/equaldex...          0.050000   
70  http://www.programmableweb.com/mashup/bookstea...          0.055556   
71     http://www.programmableweb.com/mashup/ibeeking          1.000000   
72        http://www.programmableweb.com/mashup/swyft          0.666667   
73       http://www.programmableweb.com/mashup/mashit          0.000000   
74        http://www.programmableweb.com/mashup/bagiq          0.000000   
75        http://www.programmableweb

##Evaluation results for recommendation algorithm, where weights for first two metrics is 1 and the third one (Delay Time) is 0 (Method 1,1,0)##

In [347]:
method1_1_0.columns = [['Method: lambdaSC=1, lambdaRE=1, lambdaDT=1']+['Mean Average precision' for i in range(10)],
                       ['Mashup']+['k = %s' %k for k in range(1,11)]]
method1_1_0.mean(axis=0)

Mean Average precision  k = 1     0.275182
                        k = 2     0.246972
                        k = 3     0.222994
                        k = 4     0.215256
                        k = 5     0.214968
                        k = 6     0.209599
                        k = 7     0.205567
                        k = 8     0.200977
                        k = 9     0.195354
                        k = 10    0.165344
dtype: float64

In [398]:
method1_1_0.columns = [['Method: lambdaSC=1, lambdaRE=1, lambdaDT=0']+['Average precision' for i in range(10)],
                       ['Mashup']+['k = %s' %k for k in range(1,11)]]
method1_1_0

Method: lambdaSC=1, lambdaRE=1, lambdaDT=0 Average precision  \
                                               Mashup             k = 1   
0   http://www.programmableweb.com/mashup/simple-l...          0.000000   
1         http://www.programmableweb.com/mashup/posto          1.000000   
2   http://www.programmableweb.com/mashup/geobytes...          0.000000   
3    http://www.programmableweb.com/mashup/stel-order          1.000000   
4       http://www.programmableweb.com/mashup/oznewsy          0.250000   
5   http://www.programmableweb.com/mashup/uber-cal...          0.000000   
6      http://www.programmableweb.com/mashup/dino-map          0.166667   
7   http://www.programmableweb.com/mashup/data-vis...          0.000000   
8   http://www.programmableweb.com/mashup/moves-ge...          0.500000   
9    http://www.programmableweb.com/mashup/se-ranking          0.000000   
10  http://www.programmableweb.com/mashup/pricemac...          0.166667   
11  http://www.programmableweb.com/mashup/moves-ma...          0.500000   
12    http://www.programmableweb.com/mashup/detective          0.500000   
13        http://www.programmableweb.com/mashup/zabra          0.000000   
14  http://www.programmableweb.com/mashup/idealist...          0.166667   
15  http://www.programmableweb.com/mashup/sdmx-sax...          0.333333   
16     http://www.programmableweb.com/mashup/ibeeking          1.000000   
17  http://www.programmableweb.com/mashup/click4su...          0.100000   
18  http://www.programmableweb.com/mashup/bitcoin-atm          0.250000   
19   http://www.programmableweb.com/mashup/streamboxr          0.000000   
20  http://www.programmableweb.com/mashup/grasshopper          0.000000   
21  http://www.programmableweb.com/mashup/bookafy-...          0.055556   
22    http://www.programmableweb.com/mashup/followfly          0.125000   
23        http://www.programmableweb.com/mashup/swyft          0.166667   
24  http://www.programmableweb.com/mashup/ifixit-a...          0.066667   
25  http://www.programmableweb.com/mashup/proprofs...               NaN   
26  http://www.programmableweb.com/mashup/going-ou...          0.000000   
27  http://www.programmableweb.com/mashup/magic-mo...          0.000000   
28  http://www.programmableweb.com/mashup/controlm...          0.000000   
29  http://www.programmableweb.com/mashup/equaldex...          0.250000   
..                                                ...               ...   
55       http://www.programmableweb.com/mashup/mashit          0.000000   
56  http://www.programmableweb.com/mashup/intellin...          1.000000   
57  http://www.programmableweb.com/mashup/weglore-...          0.000000   
58    http://www.programmableweb.com/mashup/local-rvs          0.250000   
59  http://www.programmableweb.com/mashup/profile-...          0.250000   
60  http://www.programmableweb.com/mashup/rainbow-...          1.000000   
61  http://www.programmableweb.com/mashup/zilyo-va...          0.125000   
62   http://www.programmableweb.com/mashup/ifixit-ios          0.200000   
63       http://www.programmableweb.com/mashup/jabify          0.200000   
64    http://www.programmableweb.com/mashup/findilike          0.250000   
65  http://www.programmableweb.com/mashup/duplicat...          0.125000   
66  http://www.programmableweb.com/mashup/weather-...          0.000000   
67   http://www.programmableweb.com/mashup/yahoo-boss          0.000000   
68     http://www.programmableweb.com/mashup/screenah          1.000000   
69  http://www.programmableweb.com/mashup/use-gift...          0.250000   
70  http://www.programmableweb.com/mashup/criminal...          0.333333   
71  http://www.programmableweb.com/mashup/itavli-i...          0.500000   
72    http://www.programmableweb.com/mashup/theonespy          0.333333   
73  http://www.programmableweb.com/mashup/magento-...          0.125000   
74  http://www.programmableweb.com/mashup/99design...          0.000000   
75  http://www.programmableweb.com/m

##Evaluation results for recommendation algorithm, where weights for last two metrics are 0 (Method 1,0,0)##

In [394]:
method1_0_0.columns = [['Method: lambdaSC=1, lambdaRE=0, lambdaDT=0']+['Mean Average precision' for i in range(10)],
                       ['Mashup']+['k = %s' %k for k in range(1,11)]]
method1_0_0.mean(axis=0)

Mean Average precision  k = 1     0.387583
                        k = 2     0.290034
                        k = 3     0.247921
                        k = 4     0.226014
                        k = 5     0.207222
                        k = 6     0.199552
                        k = 7     0.191371
                        k = 8     0.188553
                        k = 9     0.185387
                        k = 10    0.144334
dtype: float64

In [397]:
method1_0_0.columns = [['Method: lambdaSC=1, lambdaRE=0, lambdaDT=0']+['Average precision' for i in range(10)],
                       ['Mashup']+['k = %s' %k for k in range(1,11)]]
method1_0_0

Method: lambdaSC=1, lambdaRE=0, lambdaDT=0 Average precision  \
                                               Mashup             k = 1   
0     http://www.programmableweb.com/mashup/theonespy          0.333333   
1   http://www.programmableweb.com/mashup/bitcoin-atm          0.500000   
2     http://www.programmableweb.com/mashup/local-rvs          0.250000   
3   http://www.programmableweb.com/mashup/going-ou...          1.000000   
4   http://www.programmableweb.com/mashup/magic-mo...          0.111111   
5   http://www.programmableweb.com/mashup/moves-ge...          1.000000   
6   http://www.programmableweb.com/mashup/windshie...          1.000000   
7   http://www.programmableweb.com/mashup/uber-cal...          0.000000   
8   http://www.programmableweb.com/mashup/weglore-...          0.333333   
9   http://www.programmableweb.com/mashup/99design...          1.000000   
10  http://www.programmableweb.com/mashup/online-b...          0.000000   
11  http://www.programmableweb.com/mashup/criminal...          0.111111   
12  http://www.programmableweb.com/mashup/collecti...          0.083333   
13    http://www.programmableweb.com/mashup/detective          0.500000   
14  http://www.programmableweb.com/mashup/gemerald...          0.333333   
15  http://www.programmableweb.com/mashup/profile-...          0.000000   
16      http://www.programmableweb.com/mashup/oznewsy          1.000000   
17  http://www.programmableweb.com/mashup/bitcoin-...          0.333333   
18  http://www.programmableweb.com/mashup/pricezom...          0.166667   
19        http://www.programmableweb.com/mashup/swyft          0.500000   
20   http://www.programmableweb.com/mashup/stel-order          1.000000   
21  http://www.programmableweb.com/mashup/zilyo-va...          0.333333   
22  http://www.programmableweb.com/mashup/anything...          1.000000   
23   http://www.programmableweb.com/mashup/touristica          0.100000   
24   http://www.programmableweb.com/mashup/ifixit-ios          0.333333   
25   http://www.programmableweb.com/mashup/coopcities          0.041667   
26  http://www.programmableweb.com/mashup/intellin...          1.000000   
27  http://www.programmableweb.com/mashup/geobytes...          0.083333   
28  http://www.programmableweb.com/mashup/idealist...          1.000000   
29    http://www.programmableweb.com/mashup/athenaeum          0.666667   
..                                                ...               ...   
55  http://www.programmableweb.com/mashup/magento-...          0.125000   
56  http://www.programmableweb.com/mashup/bookafy-...          0.055556   
57   http://www.programmableweb.com/mashup/streamboxr          0.000000   
58  http://www.programmableweb.com/mashup/bookstea...          0.055556   
59  http://www.programmableweb.com/mashup/grasshopper          0.000000   
60  http://www.programmableweb.com/mashup/moves-el...          1.000000   
61  http://www.programmableweb.com/mashup/meaningc...          0.033333   
62  http://www.programmableweb.com/mashup/duplicat...          0.166667   
63     http://www.programmableweb.com/mashup/ibeeking          0.250000   
64  http://www.programmableweb.com/mashup/ifixit-a...          0.111111   
65  http://www.programmableweb.com/mashup/controlm...          0.000000   
66       http://www.programmableweb.com/mashup/mashit          1.000000   
67  http://www.programmableweb.com/mashup/itavli-i...          0.500000   
68  http://www.programmableweb.com/mashup/sdmx-sax...          0.000000   
69  http://www.programmableweb.com/mashup/serendip...          1.000000   
70  http://www.programmableweb.com/mashup/moves-ma...          1.000000   
71       http://www.programmableweb.com/mashup/jabify          1.000000   
72     http://www.programmableweb.com/mashup/jobjolly          1.000000   
73  http://www.programmableweb.com/mashup/early-mo...          0.333333   
74  http://www.programmableweb.com/mashup/moves-mo...          1.000000   
75     http://www.programmableweb.co